In [5]:
import pandas as pd

In [6]:
pogoda = pd.read_csv('./weather_data/chicago 2020-01-01 to 2020-01-31.csv')
pogoda

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,...,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,chicago,2020-01-01T00:00:00,-3.2,-7.8,-6.0,80.64,0.0,0,NaN,0.0,...,1010.6,38.9,16.0,0,0.0,0,NaN,Partially cloudy,partly-cloudy-night,"72534014819,KORD,KMDW,72530094846,74466504838,..."
1,chicago,2020-01-01T01:00:00,-3.3,-8.3,-6.1,80.99,0.0,0,NaN,0.0,...,1010.7,64.4,14.7,0,0.0,0,NaN,Partially cloudy,partly-cloudy-night,"72534014819,KORD,KMDW,72530094846,74466504838,..."
2,chicago,2020-01-01T02:00:00,-3.9,-8.3,-6.6,81.23,0.0,0,NaN,0.0,...,1010.7,38.9,16.0,0,0.0,0,NaN,Partially cloudy,partly-cloudy-night,"72534014819,KORD,KMDW,72530094846,74466504838,..."
3,chicago,2020-01-01T03:00:00,-4.8,-9.1,-7.0,84.83,0.0,0,NaN,0.0,...,1011.1,18.0,16.0,0,0.0,0,NaN,Clear,clear-night,"72534014819,KORD,KMDW,72530094846,74466504838,..."
4,chicago,2020-01-01T04:00:00,-4.9,-9.2,-7.0,85.14,0.0,0,NaN,0.0,...,1010.9,18.0,16.0,0,0.0,0,NaN,Clear,clear-night,"72534014819,KORD,KMDW,72530094846,74466504838,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,chicago,2020-01-31T19:00:00,2.2,-1.0,-2.0,73.89,0.0,0,NaN,0.0,...,1021.0,76.1,15.8,0,0.0,0,NaN,Partially cloudy,partly-cloudy-night,"72534014819,KORD,KMDW,72530094846,74466504838,..."
740,chicago,2020-01-31T20:00:00,1.9,-0.9,-2.0,75.30,0.0,0,NaN,0.0,...,1021.0,100.0,14.7,0,0.0,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,..."
741,chicago,2020-01-31T21:00:00,1.9,-1.2,-2.0,75.30,0.0,0,NaN,0.0,...,1020.6,100.0,14.7,0,0.0,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,..."
742,chicago,2020-01-31T22:00:00,1.9,-1.4,-2.0,75.30,0.0,0,NaN,0.0,...,1020.3,99.3,13.8,0,0.0,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,..."


In [7]:
pogoda['windgust']

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
739   NaN
740   NaN
741   NaN
742   NaN
743   NaN
Name: windgust, Length: 744, dtype: float64

In [4]:
import os

BASE_PATH = os.getcwd()
EXTRACT_PATH = os.path.join(BASE_PATH, 'extracted.pkl')
EXTRACT_PATH

'c:\\Users\\Tomek\\Desktop\\PW MINI IAD\\Semestr 6\\Hurtownie\\ChicagoCrashesETL\\extracted.pkl'